In [1]:
from flask import Flask, render_template, request
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import pyodbc
import base64

model = load_model('./static/model.h5')  # Load pre-trained model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [2]:
server = 'azdatabas.database.windows.net'
database = 'DRZ'
username = 'ATWAN'
password = 'Mm@12345'

connection_string = 'Driver={ODBC Driver 18 for SQL Server};Server=' + server + ';Database=' + database + ';UID=' + username + ';PWD=' + password + ';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

In [3]:
COUNT = 0
app = Flask(__name__)
app.config["SEND_FILE_MAX_AGE_DEFAULT"] = 1

@app.route('/')
def man():
    return render_template('index.html')

@app.route('/home', methods=['POST'])
def home():
    # Connect to the Azure SQL Database
    Connection = pyodbc.connect(connection_string)
    cursor = Connection.cursor()

    img = request.files['image']
    img_binary = img.read()

    img_arr = cv2.imdecode(np.frombuffer(img_binary, np.uint8), -1)
    img_arr = cv2.resize(img_arr, (128, 128))
    img_arr = img_arr / 255.0
    img_arr = img_arr.reshape(1, 128, 128, 3)
    prediction = model.predict(img_arr)
    probability_class_1 = prediction[0, 0]

    # Insert image and prediction into the database
    cursor.execute("INSERT INTO ImagePredictions (image, prediction) VALUES (?, ?)",
               (pyodbc.Binary(img_binary), str(probability_class_1)))
    Connection.commit()
    Connection.close()


    # Pass prediction and image as a dictionary to the template
    return render_template('prediction.html', data={'image': base64.b64encode(img_binary).decode('utf-8'), 'prediction': probability_class_1})


In [9]:


# @app.route('/prediction_data')
# def prediction_data():
#     # Connect to the Azure SQL Database
#     Connection = pyodbc.connect(connection_string)
#     cursor = Connection.cursor()

#     # Fetch the latest image and its prediction from the database
#     cursor.execute("SELECT TOP 1 image, prediction FROM ImagePredictions ORDER BY id DESC")
#     row = cursor.fetchone()
#     image_binary = row[0]
#     prediction = row[1]

#     Connection.close()

#     # Return the image binary data and its prediction
#     return image_binary, prediction

# @app.route('/load_img')
# def load_img():
#     global COUNT
#     return send_from_directory('Data', "{}.jpg".format(COUNT - 1))

In [7]:
# COUNT = 0
# app = Flask(__name__)
# app.config["SEND_FILE_MAX_AGE_DEFAULT"] = 1

# @app.route('/')
# def man():
#     return render_template('index.html')


# @app.route('/home', methods=['POST'])
# def home():
#     global COUNT
#     img = request.files['image']

#     img.save('Data/{}.jpg'.format(COUNT))    
#     img_arr = cv2.imread('Data/{}.jpg'.format(COUNT))

#     img_arr = cv2.resize(img_arr, (128,128))
#     img_arr = img_arr / 255.0
#     img_arr = img_arr.reshape(1, 128,128,3)
#     prediction = model.predict(img_arr)

#     # Assuming the output is a probability indicating class 1
#     probability_class_1 = prediction[0, 0]
#     COUNT += 1
#     return render_template('prediction.html', data=probability_class_1)


# @app.route('/load_img')
# def load_img():
#     global COUNT
#     return send_from_directory('Data', "{}.jpg".format(COUNT-1))

In [5]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:11] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:12] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:12] "GET /style.css HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:16] "POST /home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:17] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:19] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:20] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2024 17:17:21] "GET /style.css HTTP/1.1" 404 -
